# Goals
* Predict Results of Pro matches with 70% accuracy

### Data that will be used
* match data and maps

In [12]:
import pandas as pd
import seaborn as sns
import numpy as np


data = pd.read_csv('scrapy/vlrgg/playerData/vlr.csv')

Using logistic regression to predict wins (for the first iteration)

Inputs for this model will include:
* 5 Players on team
* 5 Players on opponent team
* Agents each player is on
* Map

In [61]:
#resorting data so that it is indexed by game
refined_data = pd.DataFrame()

for match in data['matchID'].unique():
    map_names = data[data['matchID']==match]['map'].unique()
    for map_name in map_names:
        match_data = data[(data['matchID']==match)&(data['map']==map_name)]
        if len(match_data.index)!=10:
            #disregard incomplete games or games with less than 10 players
            print('Error: skipping incomplete match data')
            continue
        teams = match_data['playerTeam'].unique()
        try:
            team1players = match_data[match_data['playerTeam']==teams[0]]['playerName'].unique()
            team2players = match_data[match_data['playerTeam']==teams[1]]['playerName'].unique()
        except:
            print("Error: empty teams")
        try:
            refined_data =  refined_data.append(
                {'player1':team1players[0],
                'player2':team1players[1],
                'player3':team1players[2],
                'player4':team1players[3],
                'player5':team1players[4],
                'team': teams[0],
                'o_team': teams[1],
                'o_player1':team2players[0],
                'o_player2':team2players[1],
                'o_player3':team2players[2],
                'o_player4':team2players[3],
                'o_player5':team2players[4],
                'player1_agent':match_data[match_data['playerName']==team1players[0]]['playerAgent'].values[0],
                'player2_agent':match_data[match_data['playerName']==team1players[1]]['playerAgent'].values[0],
                'player3_agent':match_data[match_data['playerName']==team1players[2]]['playerAgent'].values[0],
                'player4_agent':match_data[match_data['playerName']==team1players[3]]['playerAgent'].values[0],
                'player5_agent':match_data[match_data['playerName']==team1players[4]]['playerAgent'].values[0],
                'o_player1_agent':match_data[match_data['playerName']==team2players[0]]['playerAgent'].values[0],
                'o_player2_agent':match_data[match_data['playerName']==team2players[1]]['playerAgent'].values[0],
                'o_player3_agent':match_data[match_data['playerName']==team2players[2]]['playerAgent'].values[0],
                'o_player4_agent':match_data[match_data['playerName']==team2players[3]]['playerAgent'].values[0],
                'o_player5_agent':match_data[match_data['playerName']==team2players[4]]['playerAgent'].values[0],
                'map': map_name,
                'result': match_data[match_data['playerName']==team1players[0]]['result'].values[0],
                'date': match_data[match_data['playerName']==team1players[0]]['date'].values[0]
                }, ignore_index=True)
        except:
            print("Error: skipping weird data")
refined_data.to_csv('refined.csv')



Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping weird data
Error: skipping weird data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match data
Error: skipping incomplete match d

IndexError: index 1 is out of bounds for axis 0 with size 1

In [65]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

used_columns = ['playerName', 'playerTeam', 'map', 'playerAgent', 'result']

lr_data = data[used_columns]
lr_data['playerAgent'][0]
data

,playerName,playerTeam,map,playerAgent,playerKills,playerDeaths,playerAssists,playerACS,playerADR,playerHS,playerFirstBlood,playerFirstDeath,matchID,opponent,result,winnerRoundsWon,loserRoundsWon,winningTeam,date,event
0,cNed,ACE,Breeze,jett,16.0,16.0,0.0,178.0,107.0,3.0,3.0,3.0,ACEvsGMB12-12-21,GMB,Lose,13,11,GMB,12-12-21,VALORANT Champions
1,Chronicle,GMB,Breeze,skye,24.0,11.0,3.0,305.0,190.0,2.0,3.0,2.0,GMBvsKRÜ11-12-21,KRÜ,Win,13,8,GMB,11-12-21,VALORANT Champions
2,zeek,ACE,Bind,raze,20.0,12.0,7.0,308.0,182.0,2.0,3.0,2.0,ACEvsTL11-12-21,TL,Win,13,6,ACE,11-12-21,VALORANT Champions
3,zeek,ACE,Breeze,kayo,13.0,17.0,9.0,170.0,121.0,0.0,3.0,0.0,ACEvsGMB12-12-21,GMB,Lose,13,11,GMB,12-12-21,VALORANT Champions
4,BONECOLD,ACE,Breeze,sova,14.0,16.0,4.0,168.0,117.0,4.0,2.0,4.0,ACEvsGMB12-12-21,GMB,Lose,13,11,GMB,12-12-21,VALORANT Champions
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140351,b3ta,SP,Bind,brimstone,17.0,20.0,8.0,217.0,147.0,1.0,0.0,1.0,EGOvsSP13-08-21,EGO,Win,13,11,SP,13-08-21,Champions Tour SEA Stage 3: Challengers Playoffs
140352,PinOcchiOs,SP,Bind,skye,15.0,20.0,6.0,174.0,102.0,3.0,3.0,3.0,EGOvsSP13-08-21,EGO,Win,13,11,SP,13-08-21,Champions Tour SEA Stage 3: Challengers Playoffs
140353,notxd,SP,Bind,viper,13.0,17.0,7.0,167.0,151.0,3.0,2.0,3.0,EGOvsSP13-08-21,EGO,Win,13,11,SP,13-08-21,Champions Tour SEA Stage 3: Challengers Playoffs
140354,RealB,EGO,Ascent,omen,11.0,14.0,3.0,120.0,89.0,4.0,1.0,4.0,BRNvsEGO14-08-21,BRN,Lose,13,9,BRN,14-08-21,Champions Tour SEA Stage 3: Challengers Playoffs


In [64]:
refined_data

,map,o_player1,o_player1_agent,o_player2,o_player2_agent,o_player3,o_player3_agent,o_player4,o_player4_agent,o_player5,...,player1_agent,player2,player2_agent,player3,player3_agent,player4,player4_agent,player5,player5_agent,result
0,Breeze,nAts,viper,Chronicle,killjoy,d3ffo,jett,sheydos,skye,Redgar,...,jett,zeek,kayo,BONECOLD,sova,starxo,viper,Kiles,cypher,"0 Lose Name: result, dtype: object"
1,Ascent,Redgar,astra,sheydos,skye,d3ffo,jett,nAts,cypher,Chronicle,...,sova,starxo,astra,cNed,jett,zeek,kayo,Kiles,killjoy,"34 Win Name: result, dtype: object"
2,Fracture,nAts,cypher,Chronicle,brimstone,d3ffo,raze,Redgar,breach,sheydos,...,astra,zeek,raze,cNed,jett,BONECOLD,breach,Kiles,cypher,"130 Lose Name: result, dtype: object"
3,Icebox,Chronicle,sova,sheydos,reyna,d3ffo,jett,nAts,viper,Redgar,...,jett,zeek,reyna,starxo,sage,Kiles,viper,BONECOLD,sova,"252 Win Name: result, dtype: object"
4,Split,d3ffo,jett,Chronicle,killjoy,nAts,viper,Redgar,astra,sheydos,...,jett,starxo,sage,zeek,raze,BONECOLD,omen,Kiles,cypher,"427 Win Name: result, dtype: object"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4401,Bind,SimoN,raze,Sephrous,killjoy,Co1d,jett,Mojo,omen,palapapapaaa,...,reyna,Meltings,jett,rude,brimstone,Flaring,sova,shiba,breach,"44407 Win Name: result, dtype: object"
4402,Split,FluX,jett,Bewbewbew,omen,JazKing,sage,zeal,cypher,McGyver,...,raze,joshh,sage,Haise,jett,bryce,omen,S1Eugene,skye,"44408 Win Name: result, dtype: object"
4403,Bind,Bewbewbew,omen,FluX,jett,JazKing,sova,McGyver,raze,zeal,...,reyna,Haise,raze,S1Eugene,sova,joshh,breach,bryce,brimstone,"44615 Win Name: result, dtype: object"
4404,Bind,dodonut,raze,Titanium,sova,nobody,brimstone,Blaize,phoenix,HaiSaShin,...,cypher,Teng,sova,Orange Panda,raze,SAGE,jett,Regret,brimstone,"44458 Win Name: result, dtype: object"
